# Problem Statement

In order to develop any drug, the first step is to do drug discovery. Now, the question arrises is that what is Drug Discovery ? Well guys, Drug Discovery is about finding valid drug molecules, refining them and selecting the good ones through some kind of algorithm, so that the best selected molecule can be used to manufacture drugs to treat diseases.

Now the problem is that there are almost $10^{60}$ drug molecules synthetically available, from which we have to first select valid molecules and do further preprocessing on them to select the good ones to develop drugs. Now, $10^{60}$ is a very huge number of molecules to search from and it is going to take lot of time to search the valid molecules. 

# What we have to do in this project ? 

In order to solve this problem, a new methodology called de novo drug design is proposed. In this methodology, molecules are generated from scratch using several generative algorithms in Deep Learning such as GANs or Sequence to Sequence Recurrent Neural Networks. These generated molecules will be valid and vastly different from those that were used as training data to train these neural networks. And that is what we will be doing in this project. 

# Step 1:
We will be training a Sequence to Sequence Recurrent Neural Network (RNN) to generate synthetic (fake, real looking and valid) molecules. 

# Step 2:
Furthermore, we will be selecting the good molecules from the valid molecules generated by Seq to Seq RNN, through some kind of an algorithm which is based on the concepts on biotechnology which is not going to be our focus (it's code will be already provided so that we don't have to think about implementing this algorithm) and we will skip implementing it's code. 

# Step 3:
Once, we get good molecules, then we use these good molecules to further fine tune the parameters of our Sequence to Sequence RNN so that it starts to generate not only valid but good molecules.

We have to keep running these 3 steps in a loop, until the Sequence to Sequence RNN starts to generate good molecules. 

# Training Data
Another question which arrises is that from where we will be getting our training data to train our Sequence to Sequence RNN. So, first of all the training dataset of 500,000 molecules is taken from the open-source ChEMBL dataset of drug-like molecules which is curated by the European Bioinformatics Institute. You can read more about this dataset here:
https://www.ebi.ac.uk/chembl/

In this dataset, the molecules were represented using SMILES (Simplified Molecular Input Line-Entry System). Using SMILES, molecules are encoded as strings. Molecules were represented using the SMILES string notation for easy interpretation by the recurrent neural network model we employ. SMILES was specifically designed with grammatical consistency and machine friendliness in mind, using characters to represent atoms, bonds, and chemical structures. For example, aromatic and aliphatic carbon atoms are represented by the symbols c and C. Single, double, and triple bonds are represented by the characters -, = ,
\#, respectively. Parenthese enclosures are used to show branches, and rings are indicated by digits immediately following the atoms where the ring is closed. The 500,000 molecules collected totalled 25 million SMILES characters. Additionally, start and end characters of “G” (go) and “\n” (new line) were appended to each molecule, yielding a total vocabulary of 53 unique characters within the dataset. All molecules were between 35 and 75 characters in length.

# Some Example Molecules and their SMILES string representations

<img src = 'https://drive.google.com/uc?id=150mi8DtiB5J1FsULIHd1Hji0EmfVM4do'>

The above picture shows SMILES string representation of two very popular drug molecules. Let's have a look on the SMILES string representation of a Drug Molecule named Epinephrine: 

# $CNC[C@H](O)c1ccc(O)c(O)c1$

In [ ]:
cd /content/drive/MyDrive

Now, in order to train our Sequence to Sequence RNN, we have to append 'G' (Go character to mark the starting of SMILES string at the extreme left most of the string and '\n' to mark the end of SMILES string at the extreme right most of the string. 

In the below code, fetching all the SMILES strings of drug molecules and appending 'G' character at the left most of the string and writing the appended strings with 'G' at the left most side, to the new file. 

In [ ]:
import numpy as np

In [ ]:
import hashlib

completed_lines_hash = set()

completed_lines_hash = set()

#Save processed data to SMILES.txt
new = open("smiles.txt", "w")

#Read in data file line by line
for line in open("data.txt", "r"):
  
    #Ensure all smiles in original data file are unique
    hashValue = hashlib.md5(line.rstrip().encode('utf-8')).hexdigest()
  
    if hashValue not in completed_lines_hash:
        completed_lines_hash.add(hashValue)
        
        #Ensure all SMILES are between 35 and 75 characters in length
        if 34 < len(line) < 75:
            #Add start token
            line = line.rjust(len(line)+1, "G")

            #Copy over SMILES satisfying requirements
            new.write(line)
    
#Close files
new.close()

In [ ]:
#Read in processed data file
data = open("smiles.txt", "r").read()

#Create a list of the unique characters in the dataset
chars = list(set(data))

#Get size (in characters) of dataset
data_size = len(data) 

#Get number of unique characters in dataset
vocab_size = len(chars)

#Print dataset properties
print("Vocab size: " + str(vocab_size))
print("Data size: " + str(data_size))
print("Characters in data: " + str(chars))

If seen carefully above then '\n' is already included in the characters in processed data without even including explicitly by us. 

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#Create array from characters in the dataset
values = np.array(chars)
print("Array of unique characters:")
print(values)

#Create unique, numerical labels for each character between 0 and n-1, where n is the number of unique characters
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print("Array of labels for each character:")
print(integer_encoded)

#Encode characters into a one-hot encoding, resulting in an array of size [num unique chars, num unique chars]
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print("Array of one-hot encoded characters:")
print(onehot_encoded)
print("Size of array of one-hot encoded characters: " + str(onehot_encoded.shape))

In [ ]:
#Read in processed data file
data = open("smiles.txt", "r").read()
#Create a list of the dataset
datalist = list(data)
#Create an array of the dataset
dataarray = np.array(datalist)
#Fit one-hot encoding to dataarray
dataarray = dataarray.reshape(len(dataarray), 1)
OHESMILES = onehot_encoder.fit_transform(dataarray).astype(int)
print("Size of one-hot encoded array of data: " + str(OHESMILES.shape))
print("One-hot encoded array of data:")
print(OHESMILES)

In [ ]:
#Save OHESMILES as a (compressed) file
np.savez_compressed("ohesmiles.npz", OHESMILES)

In [ ]:
#Create integer SMILES data
INTSMILES = [np.where(r==1)[0][0] for r in OHESMILES]

In [ ]:
#Save INTSMILES as a (compressed) file
np.savez_compressed("intsmiles.npz", INTSMILES)

In [ ]:
#Save array with SMILES character, integer encoding, and one hot encoding (vocabulary)
values = np.reshape(values, (np.shape(values)[0], 1))
vocab = np.concatenate((values, integer_encoded.astype(object)), axis = 1)
vocab = vocab[vocab[:,1].argsort()]
vocabvalues = np.reshape(vocab[:,1], (-1,1))
vocabohe = onehot_encoder.fit_transform(vocabvalues)
vocabencodings = np.concatenate((vocab, vocabohe.astype(object)), axis = 1)
print(np.shape(vocabencodings))

np.save("vocab.npy", vocabencodings)

In [ ]:
print(vocabencodings)

In [ ]:
#Load SMILES data as integer labels and as one-hot encoding
data = np.load("ohesmiles.npz")
data = data["arr_0"]

intdata = np.load("intsmiles.npz")
intdata = intdata["arr_0"]

In [ ]:
data.shape

In [ ]:
intdata.shape

In [ ]:
data = data.reshape(data.shape[0],1,data.shape[1])

In [ ]:
data.shape

In [ ]:
def training_examples_generator(sequence_length,batch_idx,batch_size):

  #Now, we will generate a batch of SMILES string representations of drug molecules as input and one time step shifted to the right representations as
  #training labels as our Generative model is RNN based therefore, the RNN variant is going to be Sequence to Sequence and hence requiring sequences as inputs
  #as well as labels

  training_features = data[int((sequence_length * batch_idx)) : int((sequence_length * batch_idx) + (sequence_length * batch_size)),:,:]
  training_labels = intdata[int((sequence_length * batch_idx)) : int((sequence_length * batch_idx) + (sequence_length * batch_size))]

  training_features_batch = np.zeros(((sequence_length-1) * batch_size, 1, data.shape[2]))
  training_labels_batch = np.zeros(((sequence_length-1) * batch_size))

  training_features_idx = 0
  training_labels_idx = 0

  for char_idx in range(sequence_length * batch_size - 1):

    if char_idx % sequence_length != (sequence_length - 1):

      #Training Features batch (does not include last character of SMILES string representation of each drug molecule in the batch)
      training_features_batch[training_features_idx,:,:] = training_features[char_idx,:,:]
      training_features_idx += 1

      if char_idx % sequence_length != 0:

        #Training Labels batch (does not include first character of SMILES string representation of each drug molecule in the batch)
        training_labels_batch[training_labels_idx] = training_labels[char_idx]
        training_labels_idx += 1

  return training_features_batch,training_labels_batch

In [ ]:
from keras import Input
from keras.layers import LSTM 
from keras.layers import Dense
from keras.models import Sequential

In [ ]:
def generative_seq2seq_model(maximum_sequence_length):

  gen_rnn_model = Sequential()

  gen_rnn_model.add(Input(shape=(maximum_sequence_length,)))
  gen_rnn_model.add(LSTM(units=1024,dropout=0.2,return_sequences=True))
  gen_rnn_model.add(LSTM(units=1024,dropout=0.2,return_sequences=True))
  gen_rnn_model.add(LSTM(units=1024,dropout=0.2,return_sequences=True))
  gen_rnn_model.add(Dense(units=data.shape[1],activation="softmax"))

  gen_rnn_model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

  return gen_rnn_model

In [ ]:
model = generative_seq2seq_model(75)

In [ ]:
model.summary()

In [ ]:
training_features_batch,training_labels_batch = training_examples_generator(75,0,128)

In [ ]:
training_features_batch.shape

In [ ]:
training_labels_batch.shape